In [26]:
import pandas as pd
import numpy as np
import sklearn
import torch.nn as nn
import sklearn.model_selection
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import pickle
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

In [27]:
def add_target(team):
    team['target'] = team['won'].shift(-1)
    return team

def rest_days(team):
    team['date_time'] = pd.to_datetime(team['date'])
    team['rest'] = (team['date_time'] - team['date_time'].shift(1)).dt.days
    team['rest'] = team['rest'].fillna(0)
    team['rest'] = team['rest'].astype(int)
    del team['date_time']
    return team

def winrate(team):
    total = team['Wins'] + team['Losses']
    total_opp = team['Wins_opp'] + team['Losses_opp']
    team['winrate'] = team['Wins'] / total
    team['winrate_opp'] = team['Wins_opp'] / total_opp
    return team

def differential(team):
    team['differential'] = team['Total'] - team['Total_opp']
    return team

def find_team_exp_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=5, adjust=False).mean()
    return rolling

def find_team_exp_average_9(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=9, adjust=False).mean()
    return rolling

def find_team_exp_average_12(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=12, adjust=False).mean()
    return rolling

def find_team_average_15(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(15).mean()
    return rolling

def find_team_average_10(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(10).mean()
    return rolling

def find_team_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(5).mean()
    return rolling

def find_team_average_3(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(3).mean()
    return rolling

def rolling(data):
    df_rolling_3 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_3 = df_rolling_3.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_3)
    df_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_5 = df_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_5)
    df_rolling_10 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_10 = df_rolling_10.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_10)
    df_rolling_15 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_15 = df_rolling_15.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_15)
    df_exp_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_5 = df_exp_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_5)
    df_exp_rolling_9 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_9 = df_exp_rolling_9.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_9)
    df_exp_rolling_12 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_12 = df_exp_rolling_12.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_12)
    exp_rolling_columns_5 = [f"{col}_exp_5" for col in df_exp_rolling_5.columns]
    exp_rolling_columns_9 = [f"{col}_exp_9" for col in df_exp_rolling_9.columns]
    exp_rolling_columns_12 = [f"{col}_exp_12" for col in df_exp_rolling_12.columns]
    rolling_columns_15 = [f"{col}_15" for col in df_rolling_15.columns]
    rolling_columns_10 = [f"{col}_10" for col in df_rolling_10.columns]
    rolling_columns_5 = [f"{col}_5" for col in df_rolling_5.columns]
    rolling_columns_3 = [f"{col}_3" for col in df_rolling_3.columns]
    df_exp_rolling_12.columns = exp_rolling_columns_12
    df_exp_rolling_9.columns = exp_rolling_columns_9
    df_exp_rolling_5.columns = exp_rolling_columns_5
    df_rolling_15.columns = rolling_columns_15
    df_rolling_10.columns = rolling_columns_10
    df_rolling_5.columns = rolling_columns_5
    df_rolling_3.columns = rolling_columns_3
    df = pd.concat([data, df_rolling_3, df_rolling_5, df_rolling_10, df_rolling_15, df_exp_rolling_5,df_exp_rolling_9, df_exp_rolling_12], axis=1)
    # df_exp_rolling_5,df_exp_rolling_9, df_exp_rolling_12
    return df

def ratio(feature):
    feature_opp = 'OPP_' + str(feature)
    free = nba[feature] / nba[feature_opp]
    return free

def ratios(nba):
    regard = []
    disregard = [col for col in nba.columns if "OPP_" in col]
    for col in disregard:
        col = col[4:100]
        if col in nba.columns:
            regard.append(col)
    nba_ratio = nba[regard].apply(ratio)
    nba_ratios_columns = [f"{col}_ratio" for col in nba_ratio.columns]
    nba_ratio.columns
    return regard

def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))

def date_change(datetime_str):
    # Parse the datetime string into a datetime object
    datetime_obj = datetime.strptime(datetime_str, '%m/%d/%Y')

    # Format the datetime object into a new string structure
    new_datetime_str = datetime_obj.strftime('%Y-%m-%d')

    return new_datetime_str

def haircut(df, date):
    df[date] = df[date].str[:10]
    return df

def convert_date_format(df):
    # Create a boolean mask to identify values in the "m/d/y" format
    mask = df['Date'].str.contains(r'\d{1,2}/\d{1,2}/\d{2}')
    
    # Apply the conversion only to values that match the mask
    df.loc[mask, 'Date'] = nba.loc[mask, 'Date'].apply(date_change)
    return df

def spread(df):
    # construct spread between home and away rankings
    ranks = [i for i in df.columns if 'RANK' in i]
    ranks_home = [i for i in ranks if '_x' in i]
    ranks_away = [i for i in ranks if '_y' in i]
    spread_columns_names = [col[:-1] for col in ranks_home]
    
    spread_columns = []
    for base_name in spread_columns_names:
        spread_column = df[f'{base_name}x'] - df[f'{base_name}y']
        spread_column.name = f'{base_name}spread'
        spread_columns.append(spread_column)
        
    spread_df = pd.concat(spread_columns, axis=1)
    return spread_df

def last_season(df):
    # construct winrate from last season
    df['last_season_winrate'] = df.groupby('Teams', 'season')['winrate'].last()

In [28]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/raw_data/NBA_2018_2024.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/data/raw_data/NBA_2018_2024.csv"

df = pd.read_csv(folder_path, index_col=0)

folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/processed_cum_2018_2024.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/processed_cumulative_season_stats_2019_2024.csv"
nba = pd.read_csv(folder_path, index_col=0)

folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/2021_01_20_onward.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/data/odds_data/2021_01_20_onward.csv"
odds = pd.read_csv(folder_path, index_col=0)

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_96573/3122641595.py:8: DtypeWarning: Columns (240) have mixed types. Specify dtype option on import or set low_memory=False.
  nba = pd.read_csv(folder_path, index_col=0)


In [29]:
df = df[df['date'] != '2024-04-02 00:00:00']

In [30]:
# nba dataframe does not include the 2018 season
df = df[~df['season'].isin([2018])]
df = df.reset_index(drop=True)
df = haircut(df, 'date')

# rename nba columns to match df
nba = haircut(nba, 'Date')
nba = convert_date_format(nba)
nba.rename(columns={'Date': 'date'}, inplace=True)

#rename odds columns to match df
odds.rename(columns={'Timestamp': 'date_next'}, inplace=True)

# construct winrate for team
df = winrate(df)
# construct differential points
df = differential(df)
# construct target
df = df.groupby("Teams", group_keys=False).apply(add_target)
# construct resting
df = df.groupby(["Teams",'season'], group_keys=False).apply(rest_days)
# games yet to play are 2
df.loc[pd.isnull(df['target']), 'target'] = 2
# convert win/loss to 1/0
df['target'] = df['target'].astype(int)

# remove metadata and target for df
removed = ['target', 'date', 'Teams_opp', 'Teams',
           'season','won', 'Wins', 'Losses', 
           'Wins_opp', 'Losses_opp']
valid_columns = df.columns[~df.columns.isin(removed)]

# scale the data for df
scaler = MinMaxScaler()
df[valid_columns] = scaler.fit_transform(df[valid_columns])

# construct rolling features to df
df = rolling(df).copy()
df = df.dropna()

# remove metadata for nba ranking 
removed = ['date', 'Teams']
valid_columns = nba.columns[~nba.columns.isin(removed)]

# scale the ranking data
scaler = MinMaxScaler()
nba[valid_columns] = scaler.fit_transform(nba[valid_columns])

# remove metadata for odds data
# removed = list(odds.columns[odds.dtypes == 'object'])
# valid_columns = odds.columns[~nba.columns.isin(removed)]

# scale the odds data
# scaler = MinMaxScaler()
# odds[valid_columns] = scaler.fit_transform(odds[valid_columns])

# construct current game metadata for df
df['home_next'] = add_col(df, 'home')
df['team_next_opp'] = add_col(df, 'Teams_opp')
df['date_next'] = add_col(df, 'date')
df = df.copy()

full = pd.merge(df, nba, on=['Teams', 'date'], how='left')

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_96573/1531626907.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Teams", group_keys=False).apply(add_target)
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_96573/1531626907.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(["Teams",'season'], group_keys=False).apply(rest_days)
/var/folders/99/zv3w

## Construction of Future Games
We need to now construct a subset of the dataframe to reflect future games.

In [31]:
mapping = {'Brooklyn Nets': 'BRK', 'Golden State Warriors': 'GSW', 'Los Angeles Lakers': 'LAL',
       'Milwaukee Bucks': 'MIL', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHO',
       'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Denver Nuggets': 'DEN',
       'Detroit Pistons': 'DET', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
       'Memphis Grizzlies': 'MEM', 'Minnesota Timberwolves': 'MIN',
       'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
       'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
       'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
       'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS',
       'Atlanta Hawks': 'ATL', 'Dallas Mavericks': 'DAL', 'LA Clippers': 'LAC', 'Miami Heat': 'MIA'}

In [32]:
# Need to update this 

future_teams = ['MIL', 'WAS', 'LAL', 'TOR','NYK', 'MIA', 'OKC','PHI', 'HOU','MIN', 'SAS', 'DEN','CLE', 'UTA', 'DAL', 'GSW']

In [33]:
def construction(df, teams):
    constructed = []
    # teams are the teams that will play in the future
    for team in teams:
        constructed.append(df[df['Teams'] == team].iloc[-1])
    tonight = pd.concat(constructed, axis=1).T
    tonight = tonight.reset_index(drop=True)
    return tonight

def opponent(tonight):
    figure = []
    for i in range(len(tonight['Teams'])):
        if (i % 2 == 0):
            figure.append(tonight['Teams'][i+1])
        else:
            figure.append(tonight['Teams'][i-1])
    return figure

In [34]:
tonight = construction(full, future_teams)

In [35]:
tonight['home_next'] = [0,1] * 8 # need to update this
tonight['team_next_opp'] = opponent(tonight)
date_to_repeat = '2024-04-02' # need to update this
tonight['date_next'] = [date_to_repeat] * len(tonight)

In [36]:
# merge stats from opposing teams
complete = tonight.merge(tonight,
               left_on=['Teams', 'date_next'],
               right_on = ['team_next_opp', 'date_next'])

# concat the spreads
spread_df = spread(complete)
complete = pd.concat([complete, spread_df], axis=1)

# concat the odds
complete = pd.merge(complete, odds, on=['Teams_x', 'date_next', 'Teams_y'], how='left')
complete = complete.dropna()

In [37]:
complete

,mp_x,fg_x,fga_x,fg%_x,3p_x,3pa_x,3p%_x,ft_x,fta_x,ft%_x,orb_x,drb_x,trb_x,ast_x,stl_x,blk_x,tov_x,pf_x,pts_x,ts%_x,efg%_x,3par_x,ftr_x,orb%_x,drb%_x,trb%_x,ast%_x,stl%_x,blk%_x,tov%_x,usg%_x,ortg_x,drtg_x,fgmaxes_x,fgamaxes_x,fg%maxes_x,3pmaxes_x,3pamaxes_x,3p%maxes_x,ftmaxes_x,ftamaxes_x,ft%maxes_x,orbmaxes_x,drbmaxes_x,trbmaxes_x,astmaxes_x,stlmaxes_x,blkmaxes_x,tovmaxes_x,pfmaxes_x,ptsmaxes_x,+/-maxes_x,ts%maxes_x,efg%maxes_x,3parmaxes_x,ftrmaxes_x,orb%maxes_x,drb%maxes_x,trb%maxes_x,ast%maxes_x,stl%maxes_x,blk%maxes_x,tov%maxes_x,usg%maxes_x,ortgmaxes_x,drtgmaxes_x,Teams_x,Total_x,Pace_x,eFG%_x,TOV%_x,ORB%_x,FT/FGA_x,ORtg_x,Wins_x,Losses_x,home_x,mp_opp_x,fg_opp_x,fga_opp_x,fg%_opp_x,3p_opp_x,3pa_opp_x,3p%_opp_x,ft_opp_x,fta_opp_x,ft%_opp_x,orb_opp_x,drb_opp_x,trb_opp_x,ast_opp_x,stl_opp_x,blk_opp_x,tov_opp_x,pf_opp_x,pts_opp_x,ts%_opp_x,efg%_opp_x,3par_opp_x,ftr_opp_x,orb%_opp_x,drb%_opp_x,trb%_opp_x,ast%_opp_x,stl%_opp_x,blk%_opp_x,tov%_opp_x,usg%_opp_x,ortg_opp_x,drtg_opp_x,fgmaxes_opp_x,fgamaxes_opp_x,fg%maxes_opp_x,3pmaxes_opp_x,3pamaxes_opp_x,3p%maxes_opp_x,ftmaxes_opp_x,ftamaxes_opp_x,ft%maxes_opp_x,orbmaxes_opp_x,drbmaxes_opp_x,trbmaxes_opp_x,astmaxes_opp_x,stlmaxes_opp_x,blkmaxes_opp_x,tovmaxes_opp_x,pfmaxes_opp_x,ptsmaxes_opp_x,+/-maxes_opp_x,ts%maxes_opp_x,efg%maxes_opp_x,3parmaxes_opp_x,ftrmaxes_opp_x,orb%maxes_opp_x,drb%maxes_opp_x,trb%maxes_opp_x,ast%maxes_opp_x,stl%maxes_opp_x,blk%maxes_opp_x,tov%maxes_opp_x,usg%maxes_opp_x,ortgmaxes_opp_x,drtgmaxes_opp_x,Teams_opp_x,Total_opp_x,Pace_opp_x,eFG%_opp_x,TOV%_opp_x,ORB%_opp_x,FT/FGA_opp_x,ORtg_opp_x,Wins_opp_x,Losses_opp_x,home_opp_x,season_x,date_x,won_x,winrate_x,winrate_opp_x,differential_x,target_x,rest_x,mp_3_x,fg_3_x,fga_3_x,fg%_3_x,3p_3_x,3pa_3_x,3p%_3_x,ft_3_x,fta_3_x,ft%_3_x,orb_3_x,drb_3_x,trb_3_x,ast_3_x,stl_3_x,blk_3_x,tov_3_x,pf_3_x,pts_3_x,ts%_3_x,efg%_3_x,3par_3_x,ftr_3_x,orb%_3_x,drb%_3_x,trb%_3_x,ast%_3_x,stl%_3_x,blk%_3_x,tov%_3_x,usg%_3_x,ortg_3_x,drtg_3_x,fgmaxes_3_x,fgamaxes_3_x,fg%maxes_3_x,3pmaxes_3_x,3pamaxes_3_x,3p%maxes_3_x,ftmaxes_3_x,ftamaxes_3_x,ft%maxes_3_x,orbmaxes_3_x,drbmaxes_3_x,trbmaxes_3_x,astmaxes_3_x,stlmaxes_3_x,blkmaxes_3_x,tovmaxes_3_x,pfmaxes_3_x,ptsmaxes_3_x,+/-maxes_3_x,ts%maxes_3_x,efg%maxes_3_x,3parmaxes_3_x,ftrmaxes_3_x,orb%maxes_3_x,drb%maxes_3_x,trb%maxes_3_x,ast%maxes_3_x,stl%maxes_3_x,blk%maxes_3_x,tov%maxes_3_x,usg%maxes_3_x,ortgmaxes_3_x,drtgmaxes_3_x,Total_3_x,Pace_3_x,eFG%_3_x,TOV%_3_x,ORB%_3_x,FT/FGA_3_x,ORtg_3_x,home_3_x,mp_opp_3_x,fg_opp_3_x,fga_opp_3_x,fg%_opp_3_x,3p_opp_3_x,3pa_opp_3_x,3p%_opp_3_x,ft_opp_3_x,fta_opp_3_x,ft%_opp_3_x,orb_opp_3_x,drb_opp_3_x,trb_opp_3_x,ast_opp_3_x,stl_opp_3_x,blk_opp_3_x,tov_opp_3_x,pf_opp_3_x,pts_opp_3_x,ts%_opp_3_x,efg%_opp_3_x,3par_opp_3_x,ftr_opp_3_x,orb%_opp_3_x,drb%_opp_3_x,trb%_opp_3_x,ast%_opp_3_x,stl%_opp_3_x,blk%_opp_3_x,tov%_opp_3_x,usg%_opp_3_x,ortg_opp_3_x,drtg_opp_3_x,fgmaxes_opp_3_x,fgamaxes_opp_3_x,fg%maxes_opp_3_x,3pmaxes_opp_3_x,3pamaxes_opp_3_x,3p%maxes_opp_3_x,ftmaxes_opp_3_x,ftamaxes_opp_3_x,ft%maxes_opp_3_x,orbmaxes_opp_3_x,drbmaxes_opp_3_x,trbmaxes_opp_3_x,astmaxes_opp_3_x,stlmaxes_opp_3_x,blkmaxes_opp_3_x,tovmaxes_opp_3_x,pfmaxes_opp_3_x,ptsmaxes_opp_3_x,+/-maxes_opp_3_x,ts%maxes_opp_3_x,efg%maxes_opp_3_x,3parmaxes_opp_3_x,ftrmaxes_opp_3_x,orb%maxes_opp_3_x,drb%maxes_opp_3_x,trb%maxes_opp_3_x,ast%maxes_opp_3_x,stl%maxes_opp_3_x,blk%maxes_opp_3_x,tov%maxes_opp_3_x,usg%maxes_opp_3_x,ortgmaxes_opp_3_x,drtgmaxes_opp_3_x,Total_opp_3_x,Pace_opp_3_x,eFG%_opp_3_x,TOV%_opp_3_x,ORB%_opp_3_x,FT/FGA_opp_3_x,ORtg_opp_3_x,home_opp_3_x,winrate_3_x,winrate_opp_3_x,differential_3_x,rest_3_x,season_3_x,mp_5_x,fg_5_x,fga_5_x,fg%_5_x,3p_5_x,3pa_5_x,3p%_5_x,ft_5_x,fta_5_x,ft%_5_x,orb_5_x,drb_5_x,trb_5_x,ast_5_x,stl_5_x,blk_5_x,tov_5_x,pf_5_x,pts_5_x,ts%_5_x,efg%_5_x,3par_5_x,ftr_5_x,orb%_5_x,drb%_5_x,trb%_5_x,ast%_5_x,stl%_5_x,blk%_5_x,tov%_5_x,usg%_5_x,ortg_5_x,drtg_5_x,fgmaxes_5_x,fgamaxes_5_x,fg%maxes_5_x,3pmaxes_5_x,3pamaxes_5_x,3p%maxes_5_x,ftmaxes_5_x,ftamaxes_5

In [21]:
# load models and features
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/factors/'
file_name = 'calibrated_predictors_ridge_classifier_30_predictors_19.97%_2019_2024.txt'
file_path = save_path + file_name
with open(file_path, 'r') as f:
    predictors = f.read()
    
# model
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/weights/'
file_name = 'calibrated_ridge_classifier_30_predictors_19.97%_2019_2024.pkl'
file_path = save_path + file_name
with open(file_path, 'rb') as f:
    clf = pickle.load(f)

predictors = predictors.split(',')[:-1]
X = complete[predictors]
predictions = clf.predict(X)
result = pd.Series(predictions, index = X.index)
final = pd.concat([complete['Teams_x'], result], axis=1)

In [22]:
probabilities = clf.predict_proba(X)[:, 1]
result = pd.DataFrame({
    'Predictions': predictions,
    'Probabilities': probabilities,
    'Fanduel': complete['Fanduel_odds_x'],
    'Draftkings': complete['Draftkings_odds_x'],
    'BetMGM': complete['BetMGM_odds_x']
}, index=X.index)

# Concatenate the predictions and probabilities with the original data
final = pd.concat([complete['date_next'],complete['Teams_x'], result], axis=1)
final.columns = ['Date','Teams', 'Predictions', 'Probabilities', 'Fanduel', 'Draftkings', 'BetMGM']
final['Our_Odds'] = 1 / final['Probabilities']
final['Best_Odds'] = final[['Fanduel', 'Draftkings', 'BetMGM']].max(axis=1)

In [23]:
from sklearn.metrics import accuracy_score

final['Actual'] = [0,1,1,0,0,1,0,1,0,1,0,1,1,0,0,1,]
accuracy = accuracy_score(final['Actual'], final['Predictions'])
accuracy

0.6875

In [24]:
final

,Date,Teams,Predictions,Probabilities,Fanduel,Draftkings,BetMGM,Our_Odds,Best_Odds,Actual
0,2024-04-02,MIL,1,1.000000,1.11,1.11,1.10,1.000000,1.11,0
1,2024-04-02,WAS,0,0.180000,7.40,7.00,7.25,5.555556,7.40,1
2,2024-04-02,LAL,1,0.785714,1.11,1.12,1.11,1.272727,1.12,1
3,2024-04-02,TOR,0,0.180000,7.10,6.75,6.75,5.555556,7.10,0
4,2024-04-02,NYK,0,0.395455,2.24,2.24,2.25,2.528736,2.25,0
5,2024-04-02,MIA,1,0.581818,1.68,1.68,1.67,1.718750,1.68,1
6,2024-04-02,OKC,1,0.620690,1.45,1.49,1.48,1.611111,1.49,0
7,2024-04-02,PHI,0,0.395455,2.80,2.70,2.70,2.528736,2.80,1
8,2024-04-02,HOU,0,0.264901,3.45,3.30,3.30,3.775000,3.45,0
9,2024-04-02,MIN,1,0.727273,1.33,1.35,1.33,1.375000,1.35,1


In [25]:
win = final[final['Predictions'] == 1]
bet = win[win['Our_Odds'] < win['Best_Odds']]
bet['odds_diff'] = bet['Best_Odds'] - bet['Our_Odds']

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_96573/3053637661.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bet['odds_diff'] = bet['Best_Odds'] - bet['Our_Odds']


In [73]:
def kelly(decimal_odds, P, fractional):
    # shrinkage is the percent we want to shrink bets to account for uncertanity
    B = decimal_odds - 1
    Q = 1 - P
    solution = (B*P - Q) / B
    solution = solution * fractional
    return solution

In [74]:
bet['Kelly'] = kelly(bet['Best_Odds'], bet['Probabilities'], 0.5)

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_85291/2365977069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bet['Kelly'] = kelly(bet['Best_Odds'], bet['Probabilities'], 0.5)


In [78]:
bet_very_undervalued = bet[bet['odds_diff'] > 0.06]
bet_very_undervalued = bet_very_undervalued[bet_very_undervalued['Best_Odds'] > 1.1]

In [79]:
bet_very_undervalued

,Date,Teams,Predictions,Probabilities,Fanduel,Draftkings,BetMGM,Our_Odds,Best_Odds,odds_diff,Kelly
0,2024-04-02,MIL,1,1.000000,1.11,1.11,1.10,1.00000,1.11,0.11000,0.500000
14,2024-04-02,DAL,1,0.543478,1.96,1.91,1.95,1.84000,1.96,0.12000,0.033967
15,2024-04-02,GSW,1,0.581818,1.89,1.91,1.87,1.71875,1.91,0.19125,0.061139
